## Introducing the `ModelData` class

This notebook introduces basic features of the `ModelData` class of pyaerocom.

In [2]:
import warnings
warnings.filterwarnings('ignore')
from pyaerocom import ModelData

Let's get a test file to load

In [7]:
from pyaerocom.test_files import get

test_files = get()
print(test_files["models"])

OrderedDict([('aatsr_su_v4.3', '/lustre/storeA/project/aerocom/aerocom-users-database/CCI-Aerosol/CCI_AEROSOL_Phase2/AATSR_SU_v4.3/renamed/aerocom.AATSR_SU_v4.3.daily.od550aer.2008.nc')])


The dictionary, returned by the `get()` function is categorised into two subirectories for model based test files (`key="models"`) and for observation based test files (`key=observations`). So far, there is not much in there (only one test file from the [Aerosol CCI project](http://www.esa-aerosol-cci.org/) but that should suffice here.

In [12]:
fpath = test_files["models"]["aatsr_su_v4.3"]

data = ModelData(fpath)

The `ModelData` class is largely based on (**but not inherited from**) the [`Cube` class from the iris project](http://scitools.org.uk/iris/docs/v1.9.0/html/iris/iris/cube.html#iris.cube.Cube) and has the nice feature (opposite to the iris `Cube` class, that it can be directly instatiated using a file-path and a valid constraint (instance of `iris.Constraint`, 